In [ ]:
import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings("ignore")

df_historical_data = pd.read_excel("Inventory Value Extract.xlsx")

In [2]:
# Convert the 'LOAD_DATE' column to datetime in one step
df_historical_data['LOAD_DATE'] = pd.to_datetime(df_historical_data['LOAD_DATE'], format="%d-%b-%y")

# Extract the month/year and day components
df_historical_data['LOAD_MON_YR'] = df_historical_data['LOAD_DATE'].dt.strftime('%Y-%m')
df_historical_data['LOAD_DAY'] = df_historical_data['LOAD_DATE'].dt.day

# Display the DataFrame information to verify the data types
df_historical_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63195 entries, 0 to 63194
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   MATERIAL             63195 non-null  object        
 1   LOAD_DATE            63195 non-null  datetime64[ns]
 2   END_DATE             63195 non-null  object        
 3   TOTAL_INV_VALUE_USD  63195 non-null  float64       
 4   PLANT                63195 non-null  int64         
 5   CURRENT_RECORD       63195 non-null  object        
 6   LOAD_MON_YR          63195 non-null  object        
 7   LOAD_DAY             63195 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(1), object(4)
memory usage: 3.6+ MB


In [3]:
# Group by plant and material and find max date values
idx = df_historical_data.groupby(['PLANT', 'MATERIAL', 'LOAD_MON_YR'])['LOAD_DAY'].idxmax()
grouped_df_historical_data = df_historical_data.loc[idx].reset_index(drop=True)

# Drop unnecessary columns
grouped_df_historical_data = grouped_df_historical_data.drop(columns=['LOAD_DAY', 'LOAD_MON_YR', 'END_DATE', 'CURRENT_RECORD'])

# Create a combined 'MATERIAL_PLANT' column
grouped_df_historical_data['MATERIAL_PLANT'] = grouped_df_historical_data['MATERIAL'] + '_' + grouped_df_historical_data['PLANT'].astype(str)

end_date = '2025-08-28'

# Set 'LOAD_DATE' as the index for time-series operations
grouped_df_historical_data = grouped_df_historical_data.set_index('LOAD_DATE').copy()

# Group the DataFrame by 'MATERIAL_PLANT', reindex, and ffill
def reindex_and_fill(group):
    # This correctly creates a date range from the group's earliest date to the end_date.
    idx = pd.date_range(start=group.index.min(), end=end_date)
    return group.reindex(idx).ffill()

filled_data = grouped_df_historical_data.groupby('MATERIAL_PLANT').apply(reindex_and_fill)

# FIX: Drop the top level of the MultiIndex created by apply()
# This removes the 'MATERIAL_PLANT' index level
filled_data = filled_data.droplevel(0)

# Now, reset the index to turn the 'LOAD_DATE' index into a column
filled_data = filled_data.reset_index()

# Rename the newly created date column explicitly
filled_data = filled_data.rename(columns={'index': 'LOAD_DATE'})

# Create the year_month and day columns on the full dataset
filled_data['year_month'] = filled_data['LOAD_DATE'].dt.strftime('%Y-%m')
filled_data['day'] = filled_data['LOAD_DATE'].dt.day

# Group by 'MATERIAL_PLANT' and 'year_month' to find the max day
#new_df = filled_data.groupby(['MATERIAL_PLANT', 'year_month']).max('day').reset_index()


# Group by 'MATERIAL_PLANT' and 'year_month', and find the index of the max 'day'
idx = filled_data.groupby(['MATERIAL_PLANT', 'year_month'])['day'].idxmax()

# Use the indices found with idxmax() to get the rows with the max day
# Select the desired columns: 'MATERIAL_PLANT', 'year_month', and the inventory value
new_df = filled_data.loc[idx, ['MATERIAL_PLANT', 'year_month', 'TOTAL_INV_VALUE_USD']].reset_index(drop=True)


Month_Year_Inventory_aggregate = new_df.groupby('year_month').sum('TOTAL_INV_VALUE_USD')
Month_Year_Inventory_aggregate.to_excel('Month_Year_Inventory_aggregate.xlsx')
new_df.to_excel('historical_inventory_output.xlsx')

C:\Users\ti290f\AppData\Local\Temp\ipykernel_14796\2113482122.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filled_data = grouped_df_historical_data.groupby('MATERIAL_PLANT').apply(reindex_and_fill)
